# A demo building your own Model Service with Awadb and langchain

In [1]:
import os, sys, torch
from transformers import AutoTokenizer, AutoModelForCausalLM, AutoModel
from langchain import HuggingFacePipeline, ConversationChain
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import AwaDB
from langchain.document_loaders import TextLoader
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.embeddings.huggingface import HuggingFaceEmbeddings

Project_path = "Input Your Project path"

In [26]:
load_type = torch.float16
if torch.cuda.is_available():
    device = torch.device(0)
else:
    device = torch.device('cpu')
# Add your local knowledge files
file_path=f"{Project_path}/knowledge.txt"
loader = TextLoader(file_path)
documents = loader.load()

# split text into sentences and embedding the sentences
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=100, chunk_overlap=50,keep_separator=False)
texts = text_splitter.split_documents(documents)
for text in texts: print(text.page_content)

embedding_path = "GanymedeNil/text2vec-large-chinese"
embeddings = HuggingFaceEmbeddings(model_name=embedding_path)
docsearch = AwaDB.from_documents(texts, embeddings)

awadb 是一款为大语言模型LLM打造的向量数据库，用于存储和快速搜索模型编码后的向量，帮助打造基于个人知识库的大模型应用
awadb 支持OpenAI, Llama, Vicuna, Alpaca, ChatGLM, Dolly等模型，以及LangChain库
awadb 是基于C语言开发的，并提供python接口，可以直接通过pip安装


In [25]:
text.page_content

'awadb 是基于C语言开发的，并提供python接口，可以直接通过pip安装'

In [ ]:
# Load LLM models, like OpenAI, Llama, Vicuna, Alpaca, ChatGLM, Dolly...
model_path = "Your own model path"
model = HuggingFacePipeline.from_model_id(model_id=model_path,
        task="text-generation",
        model_kwargs={
                      "torch_dtype" : load_type,
                      "low_cpu_mem_usage" : True,
                      "temperature": 0.2,
                      "max_length": 1000,
                      "device_map": "auto",
                      "repetition_penalty":1.1}
        )

## Query without knowledge

In [16]:
conversation = ConversationChain(llm=model, verbose=True)
question = "你好, 你知道awadb什么吗"
output = conversation.predict(input=question)
print(f"Human: {question}\nAI: {output}")

Human: 你好, 你知道awadb什么吗
AI:  是的, awadb是一个阿拉伯语单词,意思是“幸福”。


## Query with knowledge

In [34]:
prompt_template = ("基于以下信息，尽可能详细的来回答用户的问题。 "
                   "背景信息:\n{context}\n 回答用户这个问题:{question}\n\n")
PROMPT = PromptTemplate(
            template=prompt_template, input_variables=["context", "question"]
        )

chain_type_kwargs = {"prompt": PROMPT}
qa = RetrievalQA.from_chain_type(
            llm=model,
            chain_type="stuff", 
            retriever=docsearch.as_retriever(search_kwargs={"k": 3}), 
            chain_type_kwargs=chain_type_kwargs)
output = qa.run(question)


In [41]:
docs = [doc.page_content for doc in qa._get_docs(question)]
new_query = f"\n基于以下信息，尽可能详细的来回答用户的问题。 背景信息:\n{''.join(docs)}\n回答用户这个问题:{question}\n\n"
print(f"Human: {question}\n\nNew Query:{new_query}\nAI: {output}")

Human: 你好, 你知道awadb什么吗

New Query:
基于以下信息，尽可能详细的来回答用户的问题。 背景信息:
awadb 是一款为大语言模型LLM打造的向量数据库，用于存储和快速搜索模型编码后的向量，帮助打造基于个人知识库的大模型应用awadb 是基于C语言开发的，并提供python接口，可以直接通过pip安装awadb 支持OpenAI, Llama, Vicuna, Alpaca, ChatGLM, Dolly等模型，以及LangChain库
回答用户这个问题:你好, 你知道awadb什么吗


AI: awadb 是一个用于存储和快速搜索向量编码后的模型，它可以帮助打造基于个人知识库的大模型应用。它支援OpenAI、Llama、Vicuna、Alpaca、ChatGLM、Dolly等模型，以及LangChain库。
